# Find ERA5 coordinates from bounding box
Script to automatically find the bounding box for a given shapefile and round this to 0.25 degrees for use in the ERA5 downloader.

In [3]:
import math
import geopandas as gpd
from pathlib import Path

#### Control file handling

In [4]:
# Easy access to control file folder
controlFolder = Path('../0_controlFiles')

In [5]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [6]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    for line in open(file):
        
        # ... find the line with the requested setting
        if setting in line:
            break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
    
    # Return this value    
    return substring

In [7]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find spatial domain as bounding box of shapefile

In [8]:
# function to round coordinates of a bounding box to ERA5s 0.25 degree resolution
def round_coords_to_ERA5(coords):
    
    '''Assumes coodinates are an array: [lon_min,lat_min,lon_max,lat_max].
    Returns separate lat and lon vectors.'''
    
    # Extract values
    lon = [coords[0],coords[2]]
    lat = [coords[1],coords[3]]
    
    # Round to ERA5 0.25 degree resolution
    rounded_lon = [math.floor(lon[0]*4)/4, math.ceil(lon[1]*4)/4]
    rounded_lat = [math.floor(lat[0]*4)/4, math.ceil(lat[1]*4)/4]
    
    return rounded_lat, rounded_lon

In [15]:
# Find name and location of catchment shapefile
shp_path = read_from_control(controlFolder/controlFile, 'catchment_shp_path')
shp_name = read_from_control(controlFolder/controlFile, 'catchment_shp_name')

In [16]:
# Specify default path if needed
if shp_path == 'default':
    shp_path = make_default_path('shapefiles/catchment')

In [17]:
# Open the shapefile
shp = gpd.read_file(shp_path/shp_name)

In [19]:
# Get the latitude and longitude of the bounding box
bounding_box = shp.total_bounds

In [20]:
# Find the rounded bounding box
lat,lon = round_coords_to_ERA5(bounding_box)

In [24]:
# Print in ERA5 format
print('Specify coordinates as {}/{}/{}/{} in control file.'.format(lat[1],lon[0],lat[0],lon[1]))

Specify coordinates as 51.75/-116.75/50.75/-115.5 in control file.
